In [ ]:
# !pip install matplotlib
# !pip install numpy
# !pip install torch
# !pip install tqdm
!pip install tensorboard
!pip install torchsummary

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

import numpy as np
import os
import sys
import random
import torch

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import torchsummary
from tqdm import tqdm

In [ ]:
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# Load Tensorboard
%load_ext tensorboard
#%tensorboard --logdir ./runs
%tensorboard --logdir ./runs --host localhost --port 9007
#%tensorboard --logdir ./runs --host 0.0.0.0 --port 9007
#%load_ext tensorboard %tensorboard --logdir /tf/notebooks/graphs --host 0.0.0.0

In [ ]:
# Load model
%run models.ipynb

use_gpu = torch.cuda.is_available()
print("Using GPU?", use_gpu)

device = torch.device('cuda' if use_gpu else 'cpu')
model = MaskedFaceNetResNet50V2().to(device)
#model = MaskedFaceNetBackbone(num_layers=50, drop_ratio=0.6, mode='ir').to(device)


In [ ]:
print(model)

In [ ]:
#model.eval()
print(model)

In [ ]:
def train_model(model, optimizer, loaders, writer, num_epochs=5, use_mask=False):
    train_loader, val_loader, test_loader = loaders
    
    learning_rates = (0.05, 0.01, 0.01, 1e-3, 1e-3, 1e-4, 1e-5)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        sys.stdout.flush()
        
        model.train()
        train_loss = 0
        nsamples = 0
        
        if epoch < len(learning_rates):
            print("=====> Setting learning rate:", learning_rates[epoch])
            for g in optimizer.param_groups:
                g['lr'] = learning_rates[epoch]
        
        # Use hardest loss after epoch 5
        #hardest = True if epoch > 5 else False
        hardest = False
        
        # For debugging, make sure parameters are updated each epoch
        a = list(model.parameters())[0].clone()

        with tqdm(train_loader) as t:
            for i, (examples, labels) in enumerate(t):
                t.set_description(f'Train Iter {i+1}/{len(train_loader)}')
                
                optimizer.zero_grad()
                outputs = model(examples)
                
                loss = calc_loss(outputs, labels, hardest=hardest, use_mask=use_mask)
                #loss, metrics = calc_loss(outputs, labels)
                loss.backward()
                optimizer.step()
                
                nsamples += examples.size(0)
                t.set_postfix(mb_loss=loss.item())
                #t.set_postfix(mb_loss=loss.item(), run_loss=train_loss/nsamples)
                writer.add_scalar('Loss/train', loss.item(), i + epoch * len(train_loader))

        b = list(model.parameters())[0].clone()     
        print("====> Model Parameters updated:", not torch.equal(a.data, b.data))

        eval_model(model, "Val", val_loader, writer, epoch+1, hardest=hardest, use_mask=use_mask, max_show=3)
        print()

def test_model_loss(model, loader):
    model.eval()
    test_loss = 0
    nsamples = 0
    with torch.no_grad():
        for i, (examples, labels) in enumerate(tqdm(loader)):
            outputs = model(examples)
            loss = calc_loss(outputs, labels)
            accuracy = calc_accuracy(outputs, labels)
            #loss, metric = calc_loss(outputs, mask)
            #test_loss += metric
            test_loss += loss
            nsamples += examples.size(0)
        return test_loss, nsamples

def eval_model(model, title, data_loader, writer, epoch, hardest=False, use_mask=False, max_show=10):
    #test_loss, nsamples = test_model_loss(model, data_loader)
    
    model.eval()

    loss_list = []
    accu_list = []
    batch_list = []

    with torch.no_grad():
        for i, (examples, labels) in enumerate(tqdm(data_loader)):
            outputs = model(examples)
            loss = calc_loss(outputs, labels, hardest=hardest, use_mask=use_mask)
            accuracy = calc_accuracy(outputs, labels)
            loss_list.append(loss)
            accu_list.append(accuracy)
            batch_list.append(examples.size(0))
    
    print("====> Embeddings:", outputs.shape)
    loss, accu, batch = torch.tensor(loss_list).float(), torch.tensor(accu_list).float(), torch.tensor(batch_list)
    print(f"{title} Batches: {batch.shape[0]} Examples: {batch.sum()}, Loss: {loss.mean()}, Accuracy:{accu.mean()}")


In [ ]:
def calc_loss(output, labels, hardest=False, use_mask=False):
    if use_mask:
        criterion = HardTripletLossWithMask(margin=0.2, hardest=hardest).cuda()
    else:
        criterion = HardTripletLoss(margin=0.2, hardest=hardest).cuda()

    triplet_loss = criterion(output, labels)
    return triplet_loss

In [ ]:
def pairwise_distance(embeddings, squared=False, eps=1e-16):
    """ Compute the 2D matrix of distances between all the embeddings.
    """
    cor_mat = torch.matmul(embeddings, embeddings.t())
    norm_mat = cor_mat.diag()
    distances = norm_mat.unsqueeze(1) - 2 * cor_mat + norm_mat.unsqueeze(0)
    distances = F.relu(distances)

    if not squared:
        mask = torch.eq(distances, 0.0).float()
        distances = distances + mask * eps
        distances = torch.sqrt(distances)
        distances = distances * (1.0 - mask)

    return distances

def calc_accuracy(embeddings, labels, use_mask=False):
    """Calculate model accuracy given output embeddings.
    """
    num_examples = labels.shape[0]
    ds = pairwise_distance(embeddings)
    
    # Get the indexes of the min distances excluding diagonal '0's
    m = torch.eye(num_examples).to(device) * torch.max(ds)
    closest_idx = torch.min(ds+m, dim=1)[1]
    
    num_correct = 0
    for i in range(num_examples):
        if labels[i] == labels[closest_idx[i]]:
            num_correct += 1.0
    return num_correct / num_examples


In [ ]:
def train(model, optimizer, loss_fn, dataloader, metrics, params):
    """Train the model on `num_steps` batches

    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches training data
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to training mode
    model.train()

    # summary for current training loop and a running average object for loss
    summ = []
    loss_avg = utils.RunningAverage()
    
    # Use tqdm for progress bar
    with tqdm(total=len(dataloader)) as t:
        for i, (train_batch, labels_batch) in enumerate(dataloader):
            # move to GPU if available
            if params.cuda:
                train_batch, labels_batch = train_batch.cuda(non_blocking=True), labels_batch.cuda(non_blocking=True)
            # convert to torch Variables
            train_batch, labels_batch = Variable(train_batch), Variable(labels_batch)

            # compute model output and loss
            output_batch = model(train_batch)
            loss = loss_fn(output_batch, labels_batch)

            # clear previous gradients, compute gradients of all variables wrt loss
            optimizer.zero_grad()
            loss.backward()

            # performs updates using calculated gradients
            optimizer.step()

            # Evaluate summaries only once in a while
            if i % params.save_summary_steps == 0:
                # extract data from torch Variable, move to cpu, convert to numpy arrays
                output_batch = output_batch.data.cpu().numpy()
                labels_batch = labels_batch.data.cpu().numpy()

                # compute all metrics on this batch
                summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                                 for metric in metrics}
                summary_batch['loss'] = loss.item()
                summ.append(summary_batch)

            # update the average loss
            loss_avg.update(loss.item())

            t.set_postfix(loss='{:05.3f}'.format(loss_avg()))
            t.update()

    # compute mean of all metrics in summary
    metrics_mean = {metric: np.mean([x[metric]
                                     for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v)
                                for k, v in metrics_mean.items())
    logging.info("- Train metrics: " + metrics_string)

In [ ]:
# Auto reload models like face_loader
%load_ext autoreload
%autoreload 2
#from models import face_loader

In [ ]:
# Load face_loader.ipynb
%run face_loader.ipynb

In [ ]:
# Load loss_function.ipynb
%run loss_function.ipynb

In [ ]:
def main():
    use_gpu = torch.cuda.is_available()
    print("Using GPU?", use_gpu)
    
    # Load data
    #datasets, loaders = load_data(use_gpu=use_gpu)
    datasets, loaders = load_data(use_mask=False, label_mask=False)
    train_dset, val_dset, test_dset = datasets
    train_loader, val_loader, test_loader = loaders
    
    #torchsummary.summary(model, input_size=(3, 192, 192))
    sys.stdout.flush()

    #optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    writer = SummaryWriter()
    train_model(model, optimizer, loaders, writer, num_epochs=25, use_mask=False)
    eval_model(model, "Test", test_loader, writer, epoch=11)
    writer.close()

main()


In [ ]:
# Release GPU memory when CUDA reports insufficient memory
import gc
model = None
gc.collect()